In [1]:
import pypsa, numpy as np
import os
import pandas as pd
import csv
import matplotlib.pyplot as plt
#import highspy

Several Eastern Caribbean countries with only storage and no links, one year of hourly periods, with varying loads and generation from VRE 



In [2]:
csv_folder_name =  "Eastern_CaribbeanExtendableStores_rjb-12mo_v2/"
network = pypsa.Network(csv_folder_name)
scenario = "Stores_12mo-highbattcost/"

INFO:pypsa.components:Applying weightings to all columns of `snapshot_weightings`
INFO:pypsa.io:Imported network Eastern_CaribbeanExtendableStores_rjb-12mo_v2 has buses, generators, links, loads, stores


In [ ]:
#specify that we consider all snapshots
network.lopf(network.snapshots,pyomo="True")

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using glpk


In [ ]:
pypsa.Network.export_to_csv_folder(network,scenario)

In [ ]:
#print the load active power (P) consumption
network.loads_t.p.plot(figsize = (15,7))

In [ ]:
#print the generator active power (P) dispatch
network.generators_t.p.plot(figsize = (15,7))
network.generators.p_nom_opt

In [ ]:
df_cap=pd.DataFrame(network.generators.p_nom_opt)
df_cap = df_cap.reset_index()

In [ ]:
df_cap.columns = ['name','Capacity']
new = df_cap["name"].str.split(" ", n = 1, expand = True)
df_cap["Technology"]= new[1]
df_cap['Country']= new[0]
df_cap.drop(columns =["name"], inplace = True)
df_cap= df_cap.pivot(index = 'Country',columns = 'Technology', values = 'Capacity')
df_cap

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_cap.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.set_title("Capacity by Technology and Country",fontsize=24)
axs.set_ylabel("Capacity [MW]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_ylabel("Country",fontsize=20) 
fig.savefig(scenario+"capacity.png", bbox_inches='tight')  

In [ ]:
df_cap_scaled = df_cap.div(df_cap.sum(axis=1), axis=0)*100
df_cap_scaled

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_cap_scaled.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.set_title("Scaled Capacity by Technology and Country",fontsize=24)
axs.set_ylabel("Scaled Capacity [%]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_ylabel("Country",fontsize=20) 
axs.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
fig.savefig(scenario+"scaled_capacity.png", bbox_inches='tight')

In [ ]:
#network.storage_units.p_nom_opt
network.stores.e_nom_opt

In [ ]:
network.stores.capital_cost

In [ ]:
network.stores.e_nom_opt*network.stores.capital_cost

In [ ]:
(network.generators.p_nom_opt*network.generators.capital_cost).sum()

In [ ]:
(network.generators_t.p.sum()*network.generators.marginal_cost).sum()

In [ ]:
(network.generators_t.p.sum()).sum()

In [ ]:
network.generators_t.p.sum()

In [ ]:
df_storage_cost = pd.DataFrame(network.stores.e_nom_opt*network.stores.capital_cost)
df_storage_cost.columns = ['storage_cost']
df_storage_cost = df_storage_cost.reset_index()
df_storage_cost

In [ ]:
new = df_storage_cost["name"].str.split(" ", n = 1, expand = True)
df_storage_cost["Tech"]= new[1]
df_storage_cost['Country']= new[0]
df_storage_cost.drop(columns =["name"], inplace = True)
df_storage_cost= df_storage_cost.pivot(index = 'Country',columns = 'Tech', values = 'storage_cost')
df_storage_cost 

In [ ]:
df_cost=pd.DataFrame((network.generators.p_nom_opt*network.generators.capital_cost)+(network.generators_t.p.sum()*network.generators.marginal_cost))
df_cost.columns = ['generator_cost']
df_cost = df_cost.reset_index()
#df_cost

In [ ]:
new1 = df_cost["name"].str.split(" ", n = 1, expand = True)
df_cost["Technology"]= new1[1]
df_cost['Country']= new1[0]
df_cost.drop(columns =["name"], inplace = True)
df_cost = df_cost.pivot(index = 'Country',columns = 'Technology', values = 'generator_cost')
#extracted_col = pd.DataFrame(df_storage_cost['storage_cost'])
#extracted_col
df_cost = df_cost.join(df_storage_cost['Storage'])
df_cost


In [ ]:
df_gen=pd.DataFrame(network.generators_t.p.sum()/1000)
df_gen = df_gen.reset_index()
df_gen.columns = ['name','Electricity']
new = df_gen["name"].str.split(" ", n = 1, expand = True)
df_gen["Technology"]= new[1]
df_gen['Country']= new[0]
df_gen.drop(columns =["name"], inplace = True)
df_gen= df_gen.pivot(index = 'Country',columns = 'Technology', values = 'Electricity')
df_gen

In [ ]:
df_cost.sum(axis=1)/df_gen.sum(axis=1)/1000

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_gen.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.set_title("Generation by Technology and Country",fontsize=24)
axs.set_ylabel("Generation [GWh]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_ylabel("Country",fontsize=20) 
fig.savefig(scenario+"generation.png", bbox_inches='tight') 

In [ ]:
df_gen_scaled = df_gen.div(df_gen.sum(axis=1), axis=0)*100

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_gen_scaled.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.set_title("Scaled Generation by Technology and Country",fontsize=24)
axs.set_ylabel("Scaled Generation [%]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_ylabel("Country",fontsize=20)  
axs.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
fig.savefig(scenario+"scaled_generation.png", bbox_inches='tight') 

In [ ]:
network.generators_t.p.sum()/(network.generators_t.p.sum()).sum()

In [ ]:
network.generators_t.p.sum()/(network.generators.p_nom_opt*8760)

In [ ]:
((network.generators.p_nom_opt*network.generators.capital_cost).sum()+(network.generators_t.p.sum()*network.generators.marginal_cost).sum()+(network.stores.e_nom_opt*network.stores.capital_cost).sum()).sum()/(network.loads_t.p_set.sum()).sum()

In [ ]:
((network.generators.p_nom_opt*network.generators.capital_cost).sum()+(network.generators_t.p.sum()*network.generators.marginal_cost).sum()+(network.stores.e_nom_opt*network.stores.capital_cost).sum())/network.loads_t.p_set.sum()

In [ ]:
#print storage units
network.stores_t.e.plot(figsize = (15,7))

